Tema de Investigación: Apache Spark <br>
Profesor: Allan Berrocal <br>
Estudiantes: <br>
Carlos Espinoza <br>
Abigail Gamboa <br>
Greivin Sanchez <br>


In [10]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import SparkSession

In [9]:
spark = SparkSession.builder.appName("rf").getOrCreate()

In [11]:
dataFrame = spark.read.format("libsvm").load("data.txt")

22/06/06 21:11:23 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [12]:
dataFrame.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [13]:
(train, test) = dataFrame.randomSplit([0.7, 0.3], seed=42)

In [15]:
train.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [16]:
test.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [18]:
randomForestClass = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=20, seed = 42)

In [19]:
model = randomForestClass.fit(train)

In [20]:
predictions = model.transform(test)

In [21]:
predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [26]:
predictions.select("prediction", "label", "features").show(10)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|(692,[100,101,102...|
|       0.0|  0.0|(692,[123,124,125...|
|       0.0|  0.0|(692,[123,124,125...|
|       0.0|  0.0|(692,[124,125,126...|
|       0.0|  0.0|(692,[124,125,126...|
|       0.0|  0.0|(692,[124,125,126...|
|       0.0|  0.0|(692,[125,126,127...|
|       0.0|  0.0|(692,[126,127,128...|
|       0.0|  0.0|(692,[126,127,128...|
|       0.0|  0.0|(692,[126,127,128...|
+----------+-----+--------------------+
only showing top 10 rows



In [27]:
eval = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

In [28]:
accuracy = eval.evaluate(predictions)

In [30]:
print("Test error: %g" % (1.0 - accuracy))

Test error: 0


GBT Classifier

In [31]:
from pyspark.ml.classification import GBTClassifier

In [33]:
gradientBoostClass = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10, seed=42)

In [34]:
model = gradientBoostClass.fit(train)

In [35]:
predictions = model.transform(test)

In [36]:
predictions.select("prediction", "label", "features")

DataFrame[prediction: double, label: double, features: vector]

In [37]:
eval = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = eval.evaluate(predictions)

22/06/06 21:41:24 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/06/06 21:41:24 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


In [38]:
print("Test error: %g" % (1.0 - accuracy))

Test error: 0.0571429
